In [1]:
import selenium
import json
import nltk
import re
import string
import time
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from numpy import mean
from numpy import absolute
from numpy import sqrt
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from nltk import wordpunct_tokenize as wp
from nltk.corpus import stopwords

sWords = set(stopwords.words('spanish'))
sStemmer=SnowballStemmer("spanish")

## look up langid for large corpora

In [2]:
with open('reviews.json', 'r', encoding="utf8") as jFile:
        data = jFile.read()

In [3]:
dict1 = json.loads(data)
#Character encoding for accented characters using json.dumps() appears to be problematic
json.dumps(dict1)

'{"paper": [{"id": 1, "preliminary_decision": "accept", "review": [{"confidence": "4", "evaluation": "1", "id": 1, "lan": "es", "orientation": "0", "remarks": "", "text": "- El art\\u00edculo aborda un problema contingente y muy relevante, e incluye tanto un diagn\\u00f3stico nacional de uso de buenas pr\\u00e1cticas como una soluci\\u00f3n (buenas pr\\u00e1cticas concretas). - El lenguaje es adecuado.  - El art\\u00edculo se siente como la concatenaci\\u00f3n de tres art\\u00edculos diferentes: (1) resultados de una encuesta, (2) buenas pr\\u00e1cticas de seguridad, (3) incorporaci\\u00f3n de buenas pr\\u00e1cticas. - El orden de las secciones ser\\u00eda mejor si refleja este orden (la versi\\u00f3n revisada es #2, #1, #3). - El art\\u00edculo no tiene validaci\\u00f3n de ning\\u00fan tipo, ni siquiera por evaluaci\\u00f3n de expertos.", "timespan": "2010-07-05"}, {"confidence": "4", "evaluation": "1", "id": 2, "lan": "es", "orientation": "1", "remarks": "", "text": "El art\\u00edcul

In [4]:
for w in dict1['paper']:
    r = w.get('id')
    print('ID: ', r)
    for q in w.get('review'):
        eva = q.get('evaluation')
        print('EVAL: ', eva)
        txt = q.get('text')
        print('REVIEW TEXT: ', txt)
    print('\n')
    if r > 2:
        break

ID:  1
EVAL:  1
REVIEW TEXT:  - El artículo aborda un problema contingente y muy relevante, e incluye tanto un diagnóstico nacional de uso de buenas prácticas como una solución (buenas prácticas concretas). - El lenguaje es adecuado.  - El artículo se siente como la concatenación de tres artículos diferentes: (1) resultados de una encuesta, (2) buenas prácticas de seguridad, (3) incorporación de buenas prácticas. - El orden de las secciones sería mejor si refleja este orden (la versión revisada es #2, #1, #3). - El artículo no tiene validación de ningún tipo, ni siquiera por evaluación de expertos.
EVAL:  1
REVIEW TEXT:  El artículo presenta recomendaciones prácticas para el desarrollo de software seguro. Se describen las mejores prácticas recomendadas para desarrollar software que sea proactivo ante los ataques, y se realiza un análisis de costos de estas prácticas en desarrollo de software. Todo basado en una revisión de prácticas propuestas en la bibliografía y su contraste con dato

## Preprocessing

In [5]:
start = time.time()
##for sake of comparison, wx will be nearly identical to the multi step output below,
##only differing in that this list will not be filtered for english reviews
wx = []
for w in dict1['paper']:
    rev = w.get('review')
    for x in rev:
        t = x.get('text')
        pList = []
        revs = t.lower()
        #split tokens by whitespace
        words = revs.split()
        #remove punctuation in tokens
        table = str.maketrans('', '', string.punctuation)
        stripped = [a.translate(table) for a in words]
        stripped1 = []
        for word in stripped:
            if word:
                word = re.sub('\(', '', word)
                word = re.sub('\)', '', word)
                stripped1.append(word)
        #remove null tokens from sentences
        prSen = [y for y in stripped1 if y]
        pList.append(prSen)
        eva = x.get('evaluation')

        for s in pList:
            filterS = []
            t = ' '.join(s)
            wr = wp(t)
            for token in wr:
                if token not in sWords:
                    filterS.append(token)
            zz = []
            for x in filterS:
                y = sStemmer.stem(x)
                #zz.append(y)
                zz.append(x)
                
            zz1 = (eva, zz, prSen)    
            wx.append(zz1)
            
stop = time.time()
clock = stop - start
print(len(wx), 'articles processed', '\nruntime:', clock)
print(wx[0])

405 articles processed 
runtime: 1.130035638809204
('1', ['artículo', 'aborda', 'problema', 'contingente', 'relevante', 'incluye', 'diagnóstico', 'nacional', 'uso', 'buenas', 'prácticas', 'solución', 'buenas', 'prácticas', 'concretas', 'lenguaje', 'adecuado', 'artículo', 'concatenación', 'tres', 'artículos', 'diferentes', '1', 'resultados', 'encuesta', '2', 'buenas', 'prácticas', 'seguridad', '3', 'incorporación', 'buenas', 'prácticas', 'orden', 'secciones', 'mejor', 'si', 'refleja', 'orden', 'versión', 'revisada', '2', '1', '3', 'artículo', 'validación', 'ningún', 'tipo', 'siquiera', 'evaluación', 'expertos'], ['el', 'artículo', 'aborda', 'un', 'problema', 'contingente', 'y', 'muy', 'relevante', 'e', 'incluye', 'tanto', 'un', 'diagnóstico', 'nacional', 'de', 'uso', 'de', 'buenas', 'prácticas', 'como', 'una', 'solución', 'buenas', 'prácticas', 'concretas', 'el', 'lenguaje', 'es', 'adecuado', 'el', 'artículo', 'se', 'siente', 'como', 'la', 'concatenación', 'de', 'tres', 'artículos', 'di

Cells below demonstrate individual steps. Process continues at 'Task 1' if [wx], tokens including number tokens is used as data input. Else continue and use [yz], tokens without numer tokens.

Clean and remove whitepace

In [6]:
prList = []
for w in dict1['paper']:
    rev = w.get('review')
    for x in rev:
        t = x.get('text')
        revs = t.lower()
        #split tokens by whitespace
        words = revs.split()
        #remove punctuation in tokens
        table = str.maketrans('', '', string.punctuation)
        stripped = [a.translate(table) for a in words]
        #remove null tokens from sentences
        prSen = [y for y in stripped if y]
        eva = x.get('evaluation')
        q1 = [eva, prSen]
        prList.append(q1)
        
print(prList[0])
print(len(prList))


['1', ['el', 'artículo', 'aborda', 'un', 'problema', 'contingente', 'y', 'muy', 'relevante', 'e', 'incluye', 'tanto', 'un', 'diagnóstico', 'nacional', 'de', 'uso', 'de', 'buenas', 'prácticas', 'como', 'una', 'solución', 'buenas', 'prácticas', 'concretas', 'el', 'lenguaje', 'es', 'adecuado', 'el', 'artículo', 'se', 'siente', 'como', 'la', 'concatenación', 'de', 'tres', 'artículos', 'diferentes', '1', 'resultados', 'de', 'una', 'encuesta', '2', 'buenas', 'prácticas', 'de', 'seguridad', '3', 'incorporación', 'de', 'buenas', 'prácticas', 'el', 'orden', 'de', 'las', 'secciones', 'sería', 'mejor', 'si', 'refleja', 'este', 'orden', 'la', 'versión', 'revisada', 'es', '2', '1', '3', 'el', 'artículo', 'no', 'tiene', 'validación', 'de', 'ningún', 'tipo', 'ni', 'siquiera', 'por', 'evaluación', 'de', 'expertos']]
405


Tokenize lists

In [7]:
l2 = []
for s in prList:
    r = ' '.join(s[1])
    wr = wp(r)
    z = [s[0], wr]
    l2.append(z)
    
print(l2[0])


['1', ['el', 'artículo', 'aborda', 'un', 'problema', 'contingente', 'y', 'muy', 'relevante', 'e', 'incluye', 'tanto', 'un', 'diagnóstico', 'nacional', 'de', 'uso', 'de', 'buenas', 'prácticas', 'como', 'una', 'solución', 'buenas', 'prácticas', 'concretas', 'el', 'lenguaje', 'es', 'adecuado', 'el', 'artículo', 'se', 'siente', 'como', 'la', 'concatenación', 'de', 'tres', 'artículos', 'diferentes', '1', 'resultados', 'de', 'una', 'encuesta', '2', 'buenas', 'prácticas', 'de', 'seguridad', '3', 'incorporación', 'de', 'buenas', 'prácticas', 'el', 'orden', 'de', 'las', 'secciones', 'sería', 'mejor', 'si', 'refleja', 'este', 'orden', 'la', 'versión', 'revisada', 'es', '2', '1', '3', 'el', 'artículo', 'no', 'tiene', 'validación', 'de', 'ningún', 'tipo', 'ni', 'siquiera', 'por', 'evaluación', 'de', 'expertos']]


Remove English sentences

A number of reviews are written entirely in English.

In [8]:
print(l2[7])

['2', ['this', 'paper', 'aims', 'to', 'show', 'new', 'deployment', 'alternative', 'for', 'recommender', 'systems', 'to', 'this', 'end', 'the', 'authors', 'use', 'two', 'variations', 'chc', 'genetic', 'algorithm', 'its', 'a', 'good', 'job', 'it', 'is', 'clear', 'and', 'very', 'detailed', 'there', 'are', 'inconsistencies', 'in', 'the', 'notation', 'should', 'be', 'improved', 'the', 'formula', '7', 'has', 'a', 'notation', 'different', 'from', 'the', 'rest', 'of', 'the', 'equations', 'on', 'the', 'same', 'point', 'the', 'figure', '3', 'refers', 'to', 'perfil', 'pesos', 'euclidiana', 'and', 'vecindad', 'these', 'words', 'arent', 'related', 'with', 'the', 'text', 'i', 'think', 'that', 'the', 'results', 'shown', 'in', 'graphs', 'would', 'be', 'more', 'useful', 'if', 'they', 'are', 'shown', 'in', 'tables']]


Utilizing lang detect to remove the English reviews.

In [9]:
from langdetect import detect
l3 = [x for x in l2 if x[1]] #remove blank iterations
l4 = []
count = 1
for s in l3:
    try:
        s1 = ' '.join(s[1])
        l = detect(s1)
        if l == 'en':
            #print(count, l, 'sentences removed.')
            print('removed:', s1, '\n')
            count += 1
        else:
            l4.append(s)
        
    except: 
        print(count, 'error')
        
print(count, 'English sentences removed.\n\n')

removed: this paper aims to show new deployment alternative for recommender systems to this end the authors use two variations chc genetic algorithm its a good job it is clear and very detailed there are inconsistencies in the notation should be improved the formula 7 has a notation different from the rest of the equations on the same point the figure 3 refers to perfil pesos euclidiana and vecindad these words arent related with the text i think that the results shown in graphs would be more useful if they are shown in tables 

removed: the paper is well structured it follows a logical sequence of sections the paper does not fix to the infonor conference template the english grammar is poor it makes the paper difficult to read and follow either a rewrite the paper in spanish or b make the paper reviewed by a professional traslator most of the references are not recent 20052010 no new work since 2003 some references do not indicate year problems with equations fonts etc figures 517 are

removed: • i recommend to change the paper title in order to be according to paper ‘ body • review the english grammar some but not all of the errors founded in the review are • abstract o what does it mean that for them what does it mean that for them o … utilized to represent its appropriateness … utilized to represent their appropriateness • introducction o … has like first phase … o auscultating is it a verb • background o a systematic mapping study is a methodology that is used a systematic mapping study is a methodology used o … of existing research on performance evaluation … … of existing research on the performance evaluation … o only dieste and juristo ’ s work 10 deals only the work of dieste and juristo 10 deals • methodology o review the use of – ly with the verb to be o third question what kind of studies does use with a higher frequency certain constructs for o appropriateness of techniques review the types question structure • classification schema o in order to answer 

Utilizing NLTK to remove stopwords

In [10]:
from nltk.corpus import stopwords
sWords = set(stopwords.words('spanish'))

l5 = []
for s in l4:
    filterS = []
    regS = []
    for w in s[1]:
        regS.append(w)
        if w in sWords:
            pass
        elif w.isnumeric():
            pass
        else:
            filterS.append(w)

    y = s[0], filterS, regS
    l5.append(y)
    
for f in l5[0]:
    print(' '.join(f))

1
artículo aborda problema contingente relevante incluye diagnóstico nacional uso buenas prácticas solución buenas prácticas concretas lenguaje adecuado artículo concatenación tres artículos diferentes resultados encuesta buenas prácticas seguridad incorporación buenas prácticas orden secciones mejor si refleja orden versión revisada artículo validación ningún tipo siquiera evaluación expertos
el artículo aborda un problema contingente y muy relevante e incluye tanto un diagnóstico nacional de uso de buenas prácticas como una solución buenas prácticas concretas el lenguaje es adecuado el artículo se siente como la concatenación de tres artículos diferentes 1 resultados de una encuesta 2 buenas prácticas de seguridad 3 incorporación de buenas prácticas el orden de las secciones sería mejor si refleja este orden la versión revisada es 2 1 3 el artículo no tiene validación de ningún tipo ni siquiera por evaluación de expertos


In [11]:
print('Before stopword filter: \n\n')
count = 1
for s in l4:
    print ('Paper', count, ',', len(s[1]),'words: \n', ' '.join(s[1]), '\n')
    count += 1
    if count > 3:
        break
        
print('\nAfter stopword filter: \n\n')
count = 1
for s in l5:
    print ('Paper', count, ',', len(s[1]),'words: \n', ' '.join(s[1]), '\n')
    count += 1
    if count > 3:
        break

Before stopword filter: 


Paper 1 , 88 words: 
 el artículo aborda un problema contingente y muy relevante e incluye tanto un diagnóstico nacional de uso de buenas prácticas como una solución buenas prácticas concretas el lenguaje es adecuado el artículo se siente como la concatenación de tres artículos diferentes 1 resultados de una encuesta 2 buenas prácticas de seguridad 3 incorporación de buenas prácticas el orden de las secciones sería mejor si refleja este orden la versión revisada es 2 1 3 el artículo no tiene validación de ningún tipo ni siquiera por evaluación de expertos 

Paper 2 , 94 words: 
 el artículo presenta recomendaciones prácticas para el desarrollo de software seguro se describen las mejores prácticas recomendadas para desarrollar software que sea proactivo ante los ataques y se realiza un análisis de costos de estas prácticas en desarrollo de software todo basado en una revisión de prácticas propuestas en la bibliografía y su contraste con datos obtenidos de una 

In [12]:
#Stemming input tokens, leave one of lines 8 & 9 uncommented to run with or without stemming
yz = []
for s in l5:
    zz = []
    for x in s[1]:
        try:
            stem = sStemmer.stem(x)
            #zz.append(stem)
            zz.append(x)
        except:
            pass
    xz = (s[0], zz, s[2])  
    yz.append(xz)
    
len(yz)


382

yz & wx include evaluation, list of stemmed tokens, and list of original tokens. This data to be used for tasks 1 & 2

In [13]:
print(yz[0], '\n\n', wx[0])

('1', ['artículo', 'aborda', 'problema', 'contingente', 'relevante', 'incluye', 'diagnóstico', 'nacional', 'uso', 'buenas', 'prácticas', 'solución', 'buenas', 'prácticas', 'concretas', 'lenguaje', 'adecuado', 'artículo', 'concatenación', 'tres', 'artículos', 'diferentes', 'resultados', 'encuesta', 'buenas', 'prácticas', 'seguridad', 'incorporación', 'buenas', 'prácticas', 'orden', 'secciones', 'mejor', 'si', 'refleja', 'orden', 'versión', 'revisada', 'artículo', 'validación', 'ningún', 'tipo', 'siquiera', 'evaluación', 'expertos'], ['el', 'artículo', 'aborda', 'un', 'problema', 'contingente', 'y', 'muy', 'relevante', 'e', 'incluye', 'tanto', 'un', 'diagnóstico', 'nacional', 'de', 'uso', 'de', 'buenas', 'prácticas', 'como', 'una', 'solución', 'buenas', 'prácticas', 'concretas', 'el', 'lenguaje', 'es', 'adecuado', 'el', 'artículo', 'se', 'siente', 'como', 'la', 'concatenación', 'de', 'tres', 'artículos', 'diferentes', '1', 'resultados', 'de', 'una', 'encuesta', '2', 'buenas', 'prácticas'

## Task 1

In [14]:
### VECTORIZE THE WORDS
vect = CountVectorizer()
l6 = [' '.join(x[1]) for x in yz]
papersent_y = np.array([x[0] for x in yz])

print('shape: ', papersent_y.shape)
print(l6[0])

shape:  (382,)
artículo aborda problema contingente relevante incluye diagnóstico nacional uso buenas prácticas solución buenas prácticas concretas lenguaje adecuado artículo concatenación tres artículos diferentes resultados encuesta buenas prácticas seguridad incorporación buenas prácticas orden secciones mejor si refleja orden versión revisada artículo validación ningún tipo siquiera evaluación expertos


In [15]:
vect.fit(l6)

CountVectorizer()

In [16]:
print('vocab size {}'.format(len(vect.vocabulary_)))
print('vocab vocab content\n {}'.format(vect.vocabulary_))

vocab size 5986
vocab vocab content
 {'artículo': 489, 'aborda': 25, 'problema': 4460, 'contingente': 1254, 'relevante': 4860, 'incluye': 2950, 'diagnóstico': 1747, 'nacional': 3794, 'uso': 5737, 'buenas': 697, 'prácticas': 4585, 'solución': 5317, 'concretas': 1121, 'lenguaje': 3324, 'adecuado': 146, 'concatenación': 1096, 'tres': 5646, 'artículos': 490, 'diferentes': 1768, 'resultados': 4968, 'encuesta': 2047, 'seguridad': 5126, 'incorporación': 2972, 'orden': 4004, 'secciones': 5103, 'mejor': 3548, 'si': 5209, 'refleja': 4812, 'versión': 5836, 'revisada': 4994, 'validación': 5774, 'ningún': 3833, 'tipo': 5556, 'siquiera': 5259, 'evaluación': 2293, 'expertos': 2362, 'presenta': 4390, 'recomendaciones': 4722, 'desarrollo': 1638, 'software': 5294, 'seguro': 5127, 'describen': 1651, 'mejores': 3561, 'recomendadas': 4724, 'desarrollar': 1634, 'proactivo': 4449, 'ataques': 534, 'realiza': 4690, 'análisis': 345, 'costos': 1360, 'basado': 618, 'revisión': 5005, 'propuestas': 4548, 'bibliogra

In [17]:
bag_of_words = vect.transform(l6)
print('type: ', type(bag_of_words))
print('shape: ', bag_of_words.shape)
print('contents:\n')
print(bag_of_words)

type:  <class 'scipy.sparse.csr.csr_matrix'>
shape:  (382, 5986)
contents:

  (0, 25)	1
  (0, 146)	1
  (0, 489)	3
  (0, 490)	1
  (0, 697)	4
  (0, 1096)	1
  (0, 1121)	1
  (0, 1254)	1
  (0, 1747)	1
  (0, 1768)	1
  (0, 2047)	1
  (0, 2293)	1
  (0, 2362)	1
  (0, 2950)	1
  (0, 2972)	1
  (0, 3324)	1
  (0, 3548)	1
  (0, 3794)	1
  (0, 3833)	1
  (0, 4004)	2
  (0, 4460)	1
  (0, 4585)	4
  (0, 4812)	1
  (0, 4860)	1
  (0, 4968)	1
  :	:
  (381, 3078)	1
  (381, 3163)	1
  (381, 3804)	1
  (381, 4160)	1
  (381, 4225)	2
  (381, 4292)	1
  (381, 4390)	1
  (381, 4425)	1
  (381, 4547)	1
  (381, 4551)	1
  (381, 4595)	1
  (381, 4630)	1
  (381, 4633)	2
  (381, 4643)	1
  (381, 5155)	1
  (381, 5167)	1
  (381, 5209)	1
  (381, 5265)	1
  (381, 5529)	1
  (381, 5534)	1
  (381, 5556)	1
  (381, 5676)	2
  (381, 5678)	1
  (381, 5756)	1
  (381, 5804)	1


In [18]:
print('bag of words as an array\n{}'.format(bag_of_words.toarray()))

bag of words as an array
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]]


In [19]:
nvect = vect.get_feature_names()
nvect1 = []
for n in nvect:
    nvect1.append(n)
    
nvect1 = ', '.join(nvect1)
print('token names:\n')
print(nvect1)

token names:

10g, 16x16, 2004b, 2010b, 2º, 3d, 3er, 3era, 3rd, 4ta, 4x4news, 5o, aa, aanalizar, abarca, abarcando, abarcar, abc, abcrcpsp, abejas, abierta, abiertas, abierto, abiertos, abogo, aborda, abordada, abordadas, abordado, abordan, abordar, abordarse, abre, abreviaciones, abreviación, abreviaturas, abril, abruptamente, absoluta, absoluto, abstract, abstracta, abstrayéndome, abtract, abundantes, abuso, acabar, académica, académicas, académico, académicos, acaso, accept, acceptance, accesible, acceso, accidentes, accionable, acciones, acción, accuracy, acelerar, acento, acentos, acentuación, acentuadas, acepta, aceptable, aceptación, aceptada, aceptadas, aceptado, aceptar, aceptarlo, aceptarse, acerca, acercamiento, acercamientos, acercar, acertada, acid, acierto, aciertos, aclarado, aclarados, aclarar, aclararlo, aclararse, aclarasen, aclaratorios, acm, acmid, acomodarse, acompaña, acompañan, acompañarse, acompañe, aconseja, aconsejado, acoplado, acoplamiento, acoplamientos, ac

In [20]:
#data split into x data, bow array, and y data, sentiment value 

papersent_x = bag_of_words.toarray()
print(len(papersent_x), len(papersent_y))

382 382


In [21]:
print(type(papersent_x), type(papersent_y))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [22]:
#splitting data into train & test data
papersent_x_train = papersent_x[:-32]
papersent_x_test = papersent_x[-32:]

papersent_y_train = papersent_y[:-32]
papersent_y_test = papersent_y[-32:]

print(papersent_x_test)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]]


In [23]:
regr = linear_model.LinearRegression()

regr.fit(papersent_x_train, papersent_y_train)

LinearRegression()

In [24]:
papersent_x_test.shape

(32, 5986)

In [25]:
papersent_y_pred = regr.predict(papersent_x_test)
clist = regr.coef_
print(len(clist), 'coefficients\nmean coefficient:', (sum(clist)/len(clist)))
print('mean squared error:', mean_squared_error(papersent_y_test, papersent_y_pred))
# The coefficient of determination: 1 is perfect prediction
print('coefficient of determination: ', r2_score(papersent_y_test, papersent_y_pred))

5986 coefficients
mean coefficient: -0.0001322513161563238
mean squared error: 3.0415624873801956
coefficient of determination:  -0.6926956451507176


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


## Examples of scores optained during trials

wx with stemming:
'Coefficients: 
 [-4.63480320e+12  1.19034750e+13 -6.56638594e+12 ...  7.86169506e+09
  4.10553312e+10 -5.63027128e+10]
mean squared error: 7.694209531077907e+24
coefficient of determination:  -4.4868283370294856e+24'

wx without:
Coefficients: 
 [ 2.43547683e+11 -2.33397815e+13 -3.43869970e+12 ...  3.00531164e+10
 -3.02538765e+10  4.49328261e+10]
mean squared error: 2.8651845183046035e+24
coefficient of determination:  -1.6708137509931173e+24

yz without:
Coefficients: 
 [-0.15348371 -0.03256142 -0.02824023 ...  0.03518938  0.00909669
 -0.00702321]
mean squared error: 3.0415624873801956
coefficient of determination:  -0.6926956451507176

yz with:
Coefficients: 
 [-0.10451546 -0.04285501  0.03277861 ...  0.00523596 -0.00079409
  0.17963415]
mean squared error: 2.950436371896527
coefficient of determination:  -0.6419819808815455

Model may be suffering low r^2 due to small dataset

In [26]:
print('x shape: ', papersent_x_test.shape)
print('y shape: ', papersent_y_pred.shape)
print(papersent_y_pred)

x shape:  (32, 5986)
y shape:  (32,)
[ 1.09512777e+00  4.95656469e-01  1.33183928e+00  8.81969268e-01
  1.32039947e+00  1.68261613e+00  1.64483313e+00 -1.04789935e+00
  1.83650529e+00  2.16865308e+00  9.59709691e-01  1.79154673e+00
 -9.20038845e-01 -2.27540382e-01  1.21825039e+00 -6.42515031e-01
 -2.87264101e-01 -5.03253285e+00 -1.72842960e-01 -1.65042931e+00
  8.44040327e-01  2.50489567e-01  1.21987601e+00  1.75352112e+00
  9.27260894e-01 -3.23877191e-01  1.11950836e+00 -2.35767691e-02
  1.47956356e+00  1.60572864e+00  1.51960795e+00 -3.72340290e-03]


In [27]:
papersent_x_train[:,1].shape

(350,)

Processing time on stemmed list: 35-45s
Processing time on full list: 75-120s

In [28]:

cv = LeaveOneOut()
import time
start = time.time()
scores = cross_val_score(regr, papersent_x, papersent_y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

stop = time.time()
clock = stop - start
print('compute time in seconds: ', clock, '\nleave one out cross validation score: ', mean(absolute(scores)))

compute time in seconds:  97.96208453178406 
leave one out cross validation score:  1.2707226248606538


In [29]:
start = time.time()

scores1 = cross_val_score(regr, papersent_x, papersent_y, scoring='neg_mean_absolute_error', n_jobs=-1)

stop = time.time()
clock = stop - start
print('compute time in seconds: ', clock, '\ncross validation score: ',mean(absolute(scores1)))

compute time in seconds:  1.416092872619629 
cross validation score:  1.3001828798522872


Some examples borrowed from https://f1000research.com/for-referees/peer-reviewing-tips/examples

In [30]:
revin = ['El artículo era malo. Los hallazgos no fueron interesantes. Qué mal uso del tiempo fue esta lectura.', 'Qué revelador. Disfruté leyendo este artículo. Espero seguir investigando en este campo.', 'Este tema es nuevo para mí y no creo estar calificado para revisar dicho trabajo.', 'Este artículo fue revelador. Parecía desorganizado. El autor es un mal escritor y un buen científico.']
print('new reviews:')
for r in revin:
    print(r, '\n')

new reviews:
El artículo era malo. Los hallazgos no fueron interesantes. Qué mal uso del tiempo fue esta lectura. 

Qué revelador. Disfruté leyendo este artículo. Espero seguir investigando en este campo. 

Este tema es nuevo para mí y no creo estar calificado para revisar dicho trabajo. 

Este artículo fue revelador. Parecía desorganizado. El autor es un mal escritor y un buen científico. 



In [31]:
wz = []
for x in revin:
        prList = []

        revs = x.lower()
        #split tokens by whitespace
        words = revs.split()
        #remove punctuation in tokens
        table = str.maketrans('', '', string.punctuation)
        stripped = [a.translate(table) for a in words]
        #remove null tokens from sentences
        prSen = [y for y in stripped if y]
        prList.append(prSen)

        for s in prList:
            filterS = []
            t = ' '.join(s)
            wr = wp(t)
            for token in wr:
                if token not in sWords:
                    filterS.append(token)
            zz = []
            for x in filterS:
                y = sStemmer.stem(x)
                zz.append(x)
                
                
            wz.append(zz)

In [32]:
lq = [' '.join(x) for x in wz]
print(lq, len(l6))


['artículo malo hallazgos interesantes mal uso tiempo lectura', 'revelador disfruté leyendo artículo espero seguir investigando campo', 'tema nuevo creo calificado revisar dicho trabajo', 'artículo revelador parecía desorganizado autor mal escritor buen científico'] 382


In [34]:
print('vocab size {}'.format(len(vect.vocabulary_)))
#print('vocab vocab content\n {}'.format(vect.vocabulary_))

vocab size 5986


In [35]:
bag_of_words1 = vect.transform(lq)
print(bag_of_words1)

  (0, 489)	1
  (0, 2771)	1
  (0, 3129)	1
  (0, 3311)	1
  (0, 3447)	1
  (0, 5547)	1
  (0, 5737)	1
  (1, 489)	1
  (1, 759)	1
  (1, 2216)	1
  (1, 3335)	1
  (1, 5119)	1
  (2, 743)	1
  (2, 1374)	1
  (2, 1758)	1
  (2, 3882)	1
  (2, 4997)	1
  (2, 5495)	1
  (2, 5599)	1
  (3, 489)	1
  (3, 573)	1
  (3, 695)	1
  (3, 868)	1
  (3, 3447)	1


In [36]:
ar1 = bag_of_words1.toarray()
print(type(ar1))
print(ar1.shape)

<class 'numpy.ndarray'>
(4, 5986)


In [37]:
scores = regr.predict(ar1)
s1 = []
for score in scores:
    s2 = round(score, 4)
    s1.append(s2)


### Printing <test input sentences ***** evaluation scores, rounded to the nearest ten thousanth> 
for s, p in zip(lq, s1):
    print(s, '*****', p)

artículo malo hallazgos interesantes mal uso tiempo lectura ***** 0.9711
revelador disfruté leyendo artículo espero seguir investigando campo ***** 1.0718
tema nuevo creo calificado revisar dicho trabajo ***** 1.2323
artículo revelador parecía desorganizado autor mal escritor buen científico ***** 0.6704


In [38]:
type(vect)

sklearn.feature_extraction.text.CountVectorizer

## Task 2

printing 10 most important features from list of tuples via zip. Each tuple contains (feature name, coefficient)

In [39]:
a = sorted(list(zip(vect.get_feature_names(), regr.coef_)), key=lambda x: x[1], reverse=False)[:10]
b = sorted(list(zip(vect.get_feature_names(), regr.coef_)), key=lambda x: x[1], reverse=True)[:10]

In [40]:
print('most important negative features:')
count = 1
for z in a:
    print(count, z[0], 'cof', z[1])
    count += 1
print('\nmost important positive features:')
count = 1
for z in b:
    print(count, z[0], 'cof', z[1])
    count += 1

most important negative features:
1 existen cof -0.5856475643672974
2 nulo cof -0.5382743632087502
3 explicitas cof -0.5328413048846952
4 científicas cof -0.46548046023751233
5 científico cof -0.42868438362720895
6 sido cof -0.39537620328980067
7 utilizados cof -0.39383765888173855
8 solución cof -0.39227254662436933
9 contribución cof -0.38731215696423715
10 bases cof -0.3698755115696495

most important positive features:
1 alto cof 0.38875321543734687
2 prototipo cof 0.3374670086579039
3 algún cof 0.30779386942022424
4 mostrar cof 0.28733161973970556
5 dejaría cof 0.27591977127066836
6 tecnologías cof 0.2714068894230979
7 trata cof 0.26592835492119865
8 interesante cof 0.2599666482908676
9 buen cof 0.25471348617078743
10 gustado cof 0.24883868999282932
